In [85]:
import textdistance
import nltk
import time
from csv import writer
from csv import reader
import random
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kgana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
'''
Instructions to set up with pip
pip install textdistance
pip install nltk
pip install csv
pip install numpy
numpy and random should be installed with python
'''

'\nInstructions to set up with pip\npip install textdistance\npip install nltk\npip install csv\npip install numpy\nnumpy and random should be installed with python\n'

In [70]:
'''
#Terrible version
def doOperation(data, principal):       
    compared = []
    comparedVal = []
    
    #Checks to see what sentences have the same element_name and reg_name
    for comp in data:
        if principal[2] == comp[2] and principal[4] == comp[4]:
            compared.append(comp[1])
        
    print(len(compared))
    
    
    principal[1] = principal[1].replace("++++", " ")
    principal[1] = principal[1].replace(" .", ".")
    principal[1] = principal[1].replace("  ", " ")
    principal[1] = principal[1].replace(" ,", ",")
            
    #goes through the entire compared list to the message of curr and conducts a syntactical similarity test
    for idx,item in enumerate(compared):
        item = item.replace("++++", " ")
        item = item.replace(" .", ".")
        item = item.replace("  ", " ")
        item = item.replace(" ,", ",")
        tokens_1 = principal[1].split()
        tokens_2 = item.split()
        tokens_v2_1 = [word for word in tokens_1 if not word in stopwords.words()]
        tokens_v2_2 = [word for word in tokens_2 if not word in stopwords.words()]
        comparedVal.append(100 * textdistance.jaccard(tokens_v2_1 , tokens_v2_2))

    finalVals = comparedVal
    compFinal = compared
    finalVals.sort(reverse = True)

    final = []
    
    for i in range (len(compared)):
        found = False
        cnt = 0;
        while found == False:
            if comparedVal[cnt] == finalVals[i]:
                if compared[cnt] != "empty":
                    found = True
                    final.append(compared[cnt])
                    compared[cnt] = "empty"
            cnt = cnt + 1

 
    This script is for printing and readability. Remove once you want to start storing the data some other way
    
    print("Number: " + str(principal[0]))
    print("Message: " + str(principal[1]))
    print("")
    print(len(final))
    print(len(finalVals))
    At this point you have finalVals and final in lists final has the sentences and finalVals has the scores and figure out how you
    you want to store the data
    

    
    for i in range (11):
        None

        add final[i] to the place to store
        add finalVals[i] to the place to store

def main():
    print("start")
    principal = []
    data = []
    # open both files and add rows to the lists above. Compared will be used later
    t0 = time.perf_counter()
    with open('data_filtered20.csv', encoding="utf8") as csv:
        csv_reader = reader(csv)
        for row in csv_reader:
            data.append(row)
    print("data finished")
    with open('principal_filtered20.csv', encoding="utf8") as csv:
        csv_reader = reader(csv)
        for row in csv_reader:
            principal.append(row)
    print("principal finished")
    for index, row in enumerate(principal):
        if index is not 0:
            t2 = time.perf_counter()
            doOperation(data,row)
            print(str(index) + " finished in " + str(t2- t0) + " seconds")
    t1 = time.perf_counter()
    print(str(t1 - t0))  
    
if __name__ == "__main__":
    main()
'''

'\n#Terrible version\ndef doOperation(data, principal):       \n    compared = []\n    comparedVal = []\n    \n    #Checks to see what sentences have the same element_name and reg_name\n    for comp in data:\n        if principal[2] == comp[2] and principal[4] == comp[4]:\n            compared.append(comp[1])\n        \n    print(len(compared))\n    \n    \n    principal[1] = principal[1].replace("++++", " ")\n    principal[1] = principal[1].replace(" .", ".")\n    principal[1] = principal[1].replace("  ", " ")\n    principal[1] = principal[1].replace(" ,", ",")\n            \n    #goes through the entire compared list to the message of curr and conducts a syntactical similarity test\n    for idx,item in enumerate(compared):\n        item = item.replace("++++", " ")\n        item = item.replace(" .", ".")\n        item = item.replace("  ", " ")\n        item = item.replace(" ,", ",")\n        tokens_1 = principal[1].split()\n        tokens_2 = item.split()\n        tokens_v2_1 = [word 

In [96]:
df_data = pd.read_csv('data_filtered20.csv') 
df_principal = pd.read_csv('principal_filtered20.csv')

In [97]:
def get_support_sentences(x):
    query = [x['sentence']]
    element_name = x['element_name']
    reg_name = x['reg_name']
    corpus = df_data.loc[(df_data['element_name']==element_name) & (df_data['reg_name']==reg_name)]
    if corpus.shape[0] >= 20:
        return get_similar(query, corpus)
    else:
        return None

In [98]:
def get_jaccard(str1, str2): 
    list1 = str1.split(" ")
    list2 = str2.split(" ")
    #tokens_v2_1 = [word for word in list1 if not word in stopwords.words()]
    #tokens_v2_2 = [word for word in list2 if not word in stopwords.words()]
    return float(100 * textdistance.jaccard(list1 , list2))

In [99]:
def get_similar(queries, corpus):   
    support_dict = {}
    closest_n = 20
    distances = []
    for query in queries:
        for sent in corpus['sentence'].tolist():
            try:
                distances = np.append(distances, get_jaccard(query,sent))
            except:
                distances = np.append(distances, 0)
        results = zip(corpus['Unnamed: 0'].tolist(), distances)
        results = sorted(results, key=lambda x: x[1],reverse=True)
        for idx, distance in results[1:closest_n]:
            support_dict[idx] = distance
    return support_dict

In [100]:
start = time.time()
print(start)
df_principal.loc[:, ('support')] = df_principal.apply(get_support_sentences, axis=1)
print(time.time() - start)

1600187435.5295107
5743.895683526993


In [103]:
df_jaccard = df_principal
df_jaccard.to_pickle("jaccard.pkl")

In [102]:
df_bert = pd.read_pickle("support_filtered.pkl")
df_bert

,Unnamed: 0,sentence,element_name,element_type,reg_name,reg_type,reg_label,support
0,274,We have recently shown that the accumulation o...,NF-kappaB,Protein Family|Protein Complex,ER,Protein,1,"{455424: 0.8564616913033709, 314382: 0.8233839..."
1,347,This means in turn reduced liberation of the p...,TNF-alpha,Protein,HMGB1,Protein,0,"{439846: 0.7776271781156937, 1774137: 0.768871..."
2,354,A more unspecific mechanism caused by its anti...,cell death,Biological Process,apoptosis,Biological Process,0,"{1251971: 0.8145650464468555, 2929397: 0.80329..."
3,356,It is proposed that a vicious circle develops ...,inflammation,Biological Process,TNF-alpha,Protein,1,"{2842255: 0.8334853837786113, 1626338: 0.80803..."
4,368,When ROS production is enhanced because of NAD...,apoptosis,Biological Process,ROS,Chemical,1,"{1307004: 0.8559755998904782, 212479: 0.845751..."
...,...,...,...,...,...,...,...,...
60836,3865632,Our speculation of the over-riding association...,proliferation,Biological Process,PD-1,Protein,1,"{2683833: 0.7854059753728427, 1180542: 0.76500..."
60837,3865636,"Moreover , tolerance against brain antigens by...",injury,Biological Process,CNS,Chemical,1,"{793897: 0.7677901419863133, 3096439: 0.758760..."
60838,3865884,"However , once the negative regulation is lost...",proliferation,Biological Process,VEGF,Protein,1,"{1926520: 0.8393916434021516, 1437321: 0.83553..."
60839,3865892,AP1 was reported to be an essential component ...,apoptosis,Biological Process,light,Protein,1,"{2407046: 0.7961081292110734, 1070327: 0.78567..."


In [ ]:
# Get a proportion to see overlap of bert and jaccard 5,10, and 20
# Attention analysis on the network